<a href="https://colab.research.google.com/github/tlsdmswn01/PyTorch_Study/blob/main/FashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as T

In [ ]:
train_transform=T.Compose([
    T.ToTensor()
])

test_transform=T.Compose([
    T.ToTensor()
])


training_data= datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=train_transform
)

test_data=datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform = test_transform
)

train_dataloader=DataLoader(training_data,batch_size=64)
test_dataloader=DataLoader(test_data,batch_size=64)

In [ ]:
image,label=next(iter(train_dataloader))
print(image.size(), label)

In [ ]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self,x):
        x=self.flatten(x)
        logits=self.linear_relu_stack(x)
        return logits

model=DNN()
model.to('cuda')

In [ ]:
# 하이퍼파라미터
learning_rate = 1e-3
batch_size=64
epochs=5
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [ ]:
len(train_dataloader.dataset)

In [ ]:
def train(train_dataloader, model, loss_fn, optimizer):
    size=len(train_dataloader.dataset)

    model.train()
    for idx, (image,label) in enumerate(train_dataloader):

        image,label=image.to('cuda'), label.to('cuda')

        outputs=model(image)
        loss=loss_fn(outputs,label)


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if idx%100==0:
            loss,current= loss.item(), (idx+1)*len(image)
            print('loss:{:7f} current: {:5d}/{:5d}'.format(loss,current,size))


In [ ]:
train(train_dataloader,model,loss_fn,optimizer)

In [ ]:
def test(test_dataloader, model, loos_fn):
    model.eval()
    size=len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    test_loss, correct=0,0

    with torch.no_grad():
        for idx, (image,label) in enumerate(test_dataloader):
            image,label=image.to('cuda'), label.to('cuda')

            outputs=model(image)
            test_loss+=loss_fn(outputs,label).item()
            correct+=(outputs.argmax(1)==label).type(torch.float).sum().item()

    test_loss/=num_batches
    correct/=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")